In [33]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

# Read the Parquet file
file_path = "/Users/Macington/Documents/Projects/Project Gambit/Games/ply.parquet"
table = pq.read_table(file_path)
df = table.to_pandas()

print(df)


                      pgn_id  ply             board_sum     progression_hash  \
0        4966373121947802697    0  18446462598732906495  6548726006382385350   
1        1613272150230853183    0  18446462598732906495  6548726006382385350   
2        1613272150230853183    1  18446462598735003583  6838202551196677307   
3         127762129624462438    0  18446462598732906495  6548726006382385350   
4         127762129624462438    1  18446462598749683455 -1959233720066286525   
...                      ...  ...                   ...                  ...   
7054364 -2014757526705662466  236  13835102769989829121 -3085291962456116356   
7054365 -2014757526705662466  237  13835102769985634817  2208772014170310088   
7054366 -2014757526705662466  238  13835102767846539777  5917023950694882328   
7054367 -2014757526705662466  239  13835102767578105345 -7270769053707336975   
7054368 -2014757526705662466  240  13835102492700198401  7625980533342292681   

         centipawn  
0             18.0